# Includes

In [ ]:
!pip install --upgrade git+https://github.com/divamgupta/image-segmentation-keras

  Cloning https://github.com/divamgupta/image-segmentation-keras to /tmp/pip-req-build-4kobmh7j
  Running command git clone -q https://github.com/divamgupta/image-segmentation-keras /tmp/pip-req-build-4kobmh7j
     |████████████████████████████████| 3.3MB 8.7MB/s 
     |████████████████████████████████| 952kB 50.6MB/s 
  Created wheel for keras-segmentation: filename=keras_segmentation-0.3.0-cp37-none-any.whl size=34545 sha256=5b2940419d17de2d182d51f7abe75e9b1234e97a9fbb379665a9ad91c156dc26
  Stored in directory: /tmp/pip-ephem-wheel-cache-drydfkuc/wheels/a8/be/2c/1e1f0a75330e3ae33dd34c2ba085929eb7c7ae1d2184f4b7b5
Successfully built keras-segmentation
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.4.0 which is incompatible.
  Found existing installation: imageio 2.4.1
    Uninstalling imageio-2.4.1:
      Successfully uninstalled imageio-2.4.1
  Found existing installation: imgaug 0.2.9
    Uninstalling imgaug-0.2.9:
      Successfully unins

In [ ]:
from keras_segmentation.models.segnet import resnet50_segnet, vgg_segnet, segnet
import tensorflow as tf
import os
import tensorflow_datasets as tfds
import cv2
import numpy as np
import glob
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Models Preparation

In [ ]:
img_path = '/content/drive/MyDrive/ResNet/data/img/'
proc_img_path = '/content/drive/MyDrive/SegNet/data/img/'
mask_path = '/content/drive/MyDrive/ResNet/data/masks_machine/'
proc_mask_path = '/content/drive/MyDrive/SegNet/data/ann/'

In [ ]:
model = segnet(n_classes=3 ,  input_height=480, input_width=640  )

In [ ]:
dirs = os.listdir(img_path)
img_dataset = []
for item in dirs:
        if os.path.isfile(img_path+item):
            im = cv2.imread(img_path+item)
            f, e = os.path.splitext(img_path+item)
            im = cv2.resize(im, (480,640))
            cv2.imwrite(proc_img_path + item, im)

In [ ]:
dirs = os.listdir(mask_path)
mask_dataset = []

for item in dirs:
        if os.path.isfile(mask_path+item):
            im = cv2.imread(mask_path+item)
            im = cv2.resize(im, (480,640))
            im[im == 3] = 0
            cv2.imwrite(proc_mask_path + item, im)

In [ ]:
model = vgg_segnet(n_classes=3 ,  input_height=480, input_width=640  )

58892288/58889256 [==============================] - 1s 0us/step


In [ ]:
model = resnet50_segnet(n_classes=3 ,  input_height=480, input_width=640  )

94658560/94653016 [==============================] - 4s 0us/step


In [ ]:
checkpoints_path = "/content/drive/MyDrive/SegNet/temp/chape_vgg_segnet_"

In [ ]:
config_file = checkpoints_path + "_config.json"
dir_name = os.path.dirname(config_file)

In [ ]:
def find_latest_checkpoint(checkpoints_path, fail_safe=True):
    latest_epoch_checkpoint = tf.train.latest_checkpoint(os.path.dirname(checkpoints_path))

    if latest_epoch_checkpoint is None:
        # This is legacy code, there should always be a "checkpoint" file in your directory

        def get_epoch_number_from_path(path):
            return path.replace(checkpoints_path, "").strip(".")

        # Get all matching files
        all_checkpoint_files = glob.glob(checkpoints_path + ".*")
        all_checkpoint_files = [ff.replace(".index", "") for ff in
                                all_checkpoint_files]  # to make it work for newer versions of keras
        # Filter out entries where the epoc_number part is pure number
        all_checkpoint_files = list(filter(lambda f: get_epoch_number_from_path(f)
                                           .isdigit(), all_checkpoint_files))
        if not len(all_checkpoint_files):
            # The glob list is empty, don't have a checkpoints_path
            if not fail_safe:
                raise ValueError("Checkpoint path {0} invalid"
                                 .format(checkpoints_path))
            else:
                return None

        # Find the checkpoint file with the maximum epoch
        latest_epoch_checkpoint = max(all_checkpoint_files,
                                      key=lambda f:
                                      int(get_epoch_number_from_path(f)))

    return latest_epoch_checkpoint

In [ ]:
out = tr.find_latest_checkpoint

In [ ]:
model_vgg.load_weights("/content/drive/MyDrive/SegNet/temp/vgg_segnet_1.3")

NameError: ignored

In [ ]:
callbacks = [
    ModelCheckpoint(
                filepath="checkpoints/" + "chape_segnet_resnet50" + ".{epoch:05d}",
                save_weights_only=True,
                verbose=True
            ),
    EarlyStopping()
]

In [ ]:
callbacks = [
    ModelCheckpoint(
                filepath="/content/drive/MyDrive/SegNet/data/checkpoint/" + model.name + ".{epoch:05d}",
                save_weights_only=True,
                verbose=True
            ),
    EarlyStopping()
]

# Train model

In [ ]:
model.load_weights("/content/drive/MyDrive/SegNet/data/checkpoint/model_3.00002")

In [ ]:
model.train(
    train_images =  proc_img_path,
    train_annotations = proc_mask_path,
    checkpoints_path = "/content/drive/MyDrive/SegNet/data/checkpoint/segnet_resnet50" , 
    epochs= 2,
    callbacks = callbacks
)

 10%|█         | 6/58 [00:00<00:01, 51.85it/s]

Verifying training dataset


100%|██████████| 58/58 [00:01<00:00, 50.49it/s]


Dataset verified! 
Epoch 1/2
512/512 [==============================] - 6484s 13s/step - loss: 0.0445 - accuracy: 0.9832

Epoch 00001: saving model to /content/drive/MyDrive/SegNet/data/checkpoint/model_3.00001
Epoch 2/2
512/512 [==============================] - 6480s 13s/step - loss: 0.0270 - accuracy: 0.9890

Epoch 00002: saving model to /content/drive/MyDrive/SegNet/data/checkpoint/model_3.00002


# Results

**segnet_resnet50:**


*   Loss: 0.0270
*   Accuracy: 0.9890

**segnet:**


*   Loss: 0.9569
*   Accuracy: 0.7218

**segnet_vgg:**


*   Loss: 
*   Accuracy: 

**segnet_resnet18:**


*   Loss: 
*   Accuracy: 

# Inference Time

In [ ]:
output_path = '/content/drive/MyDrive/SegNet/data/output/resnet50/'

In [ ]:
t = time.time()
count = 0
for img in os.listdir(img_path):
  count +=1
  model.predict_segmentation(
    inp=img_path + img,
    out_fname=output_path + img
  )
avgtime = (time.time()-t)/count

In [ ]:
print(avgtime)

2.002929531294724


In [ ]:
import time

In [ ]:
out = model.predict_segmentation(
    inp="/content/drive/MyDrive/SegNet/data/img/imagem7.png",
    out_fname="/content/drive/MyDrive/SegNet/data/output/model_3_imagem7.png"
)